<a href="https://colab.research.google.com/github/Ousman-khan/ISE-571-Assignment/blob/main/Question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pymoo --upgrade
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.variable import Real, Integer
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.selection.tournament import TournamentSelection
# Importing from the correct location in pymoo
from pymoo.termination.default import DefaultMultiObjectiveTermination


In [5]:
# Define problem class
class GateAssignmentProblem(Problem):
    def __init__(self, num_aircraft, num_gates, passengers, distances, arrival_times, departure_times):
        self.num_aircraft = num_aircraft
        self.num_gates = num_gates
        self.passengers = passengers
        self.distances = distances
        self.arrival_times = arrival_times
        self.departure_times = departure_times
        super().__init__(n_var=num_aircraft, n_obj=2, n_constr=0, xl=0, xu=num_gates-1, type_var=Integer)


In [30]:
# Define problem class
class GateAssignmentProblem(Problem):
    def __init__(self, num_aircraft, num_gates, passengers, distances, arrival_times, departure_times):
        self.num_aircraft = num_aircraft
        self.num_gates = num_gates
        self.passengers = passengers
        self.distances = distances
        self.arrival_times = arrival_times
        self.departure_times = departure_times
        # The upper bound (xu) should be num_gates - 1 to match the valid gate indices
        super().__init__(n_var=num_aircraft, n_obj=2, n_constr=0, xl=0, xu=num_gates - 1, type_var=np.int64) # Change type_var to np.int64

    def _evaluate(self, X, out, *args, **kwargs):
        f1 = np.zeros(X.shape[0])  # Passenger walking distance
        f2 = np.zeros(X.shape[0])  # Gate idling time

        for i in range(X.shape[0]):  # Loop through each solution
            gate_assignment = X[i, :].astype(int)  # Ensure gate_assignment contains integers

            # Compute total walking distance, ensuring gate_assignment[j] is within valid bounds
            f1[i] = sum(self.passengers[j] * self.distances[j, min(gate_assignment[j], self.num_gates - 1)] for j in range(self.num_aircraft))

            # Compute gate idling time
            idle_time = 0
            for g in range(self.num_gates):
                assigned_flights = [j for j in range(self.num_aircraft) if gate_assignment[j] == g]
                assigned_flights.sort(key=lambda j: self.arrival_times[j])
                for j in range(len(assigned_flights) - 1):
                    idle_time += max(0, self.arrival_times[assigned_flights[j+1]] - self.departure_times[assigned_flights[j]])
            f2[i] = idle_time

        # Scale down the walking distance to be in a similar range as the idling time
        f1 = f1 / 1000  # Dividing by 1000 as an example, adjust the scaling factor as needed

        out["F"] = np.column_stack([f1, f2])

In [31]:
# Define problem parameters
num_aircraft = 10
num_gates = 5
np.random.seed(42)
passengers = np.random.randint(50, 300, size=num_aircraft)
distances = np.random.randint(10, 100, size=(num_aircraft, num_gates))
arrival_times = np.random.randint(0, 24, size=num_aircraft)
departure_times = arrival_times + np.random.randint(1, 5, size=num_aircraft)



In [32]:
# Create problem instance
problem = GateAssignmentProblem(num_aircraft, num_gates, passengers, distances, arrival_times, departure_times)

# Configure NSGA-II algorithm
algorithm = NSGA2(
    pop_size=100,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(prob=0.2, eta=20),
    eliminate_duplicates=True
)


In [33]:
from pymoo.termination.max_gen import MaximumGenerationTermination # Import from the correct submodule
# Solve problem
res = minimize(
    problem,
    algorithm,
    termination=MaximumGenerationTermination(n_max_gen=100), # Use n_max_gen instead of n_gen
    seed=42,
    verbose=True
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      2 |             - |             -
     2 |      200 |      2 |  0.000000E+00 |             f
     3 |      300 |      3 |  0.1347911633 |             f
     4 |      400 |      8 |  0.2075858534 |         ideal
     5 |      500 |      7 |  0.0545392048 |         ideal
     6 |      600 |      4 |  0.2249929821 |         nadir
     7 |      700 |      5 |  0.000000E+00 |             f
     8 |      800 |      4 |  0.0738499728 |         ideal
     9 |      900 |      5 |  0.1666666667 |         ideal
    10 |     1000 |     10 |  0.6000000000 |         nadir
    11 |     1100 |      7 |  0.0255438036 |         ideal
    12 |     1200 |     10 |  0.0388721949 |         ideal
    13 |     1300 |      9 |  0.0338050187 |             f
    14 |     1400 |     14 |  0.0126725040 |             f
    15 |     1500 |     27 |  0.0255435798 |         ideal
    16 |     1600 |     32 |  0.0029340875 |            

In [34]:
# Print results
print("Best solutions:")
print(res.X)
print("Objective values:")
print(res.F)


Best solutions:
[[2.23829967 1.88228598 3.62969194 1.18681147 0.70929684 2.18611794
  0.06214334 2.89153787 2.7163215  4.        ]
 [2.91736384 4.         3.54775934 1.13710232 3.49662397 2.18681141
  0.56172703 2.41799089 2.98147966 0.0598782 ]
 [2.93514927 1.75806362 3.88024918 1.1548121  0.69829326 2.90639583
  0.02530641 2.04589287 2.97460417 4.        ]
 [2.79170208 4.         3.57687379 1.26833579 3.89136197 2.19667767
  0.45984756 2.09769164 2.93762047 0.23631802]
 [2.92136421 4.         3.83835961 1.17406762 0.12409223 2.14725236
  0.02928365 2.39813392 2.91333096 0.04074192]
 [2.93979126 1.74477794 3.88024918 1.         3.99730177 2.92280081
  0.54528047 2.04589287 2.97460417 4.        ]
 [2.77800752 4.         3.69161936 1.12555816 0.08081407 2.06194144
  0.04244917 2.39686791 2.74817124 0.4927486 ]
 [2.03665123 1.88228598 3.70032045 1.30019915 3.41170225 2.17218438
  0.05922572 2.14651826 2.91164896 4.        ]
 [2.92392447 4.         3.82424272 1.22927198 3.39750198 2.05545